<a href="https://colab.research.google.com/github/rohandhiman5/Kaggle-challenge-by-cynaptics-club/blob/main/CNN_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download spot-the-synthetic

 91% 29.0M/31.8M [00:00<00:00, 65.3MB/s]
100% 31.8M/31.8M [00:00<00:00, 57.7MB/s]


In [ ]:
from zipfile import ZipFile
file = 'spot-the-synthetic.zip'
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [ ]:
import os
import numpy as np
from PIL import Image

In [ ]:
def load_and_preprocess_images_from_folder(folder, target_size=(224,224)):
    images = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            img.thumbnail(target_size, Image.ANTIALIAS)
            new_img = Image.new("RGB", target_size, (255, 255, 255))
            new_img.paste(img, ((target_size[0] - img.size[0]) // 2, (target_size[1] - img.size[1]) // 2))
            img_array = np.array(new_img).astype('float32') / 255.0
            images.append(img_array)
    return np.array(images)


In [ ]:
ai_images = load_and_preprocess_images_from_folder('/content/Data/AI_Images')
real_images = load_and_preprocess_images_from_folder('/content/Data/Real_Images')
test_images = load_and_preprocess_images_from_folder('/content/Data/Test_Images')

<ipython-input-5-aa644e78db35>:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail(target_size, Image.ANTIALIAS)


In [ ]:
ai_images[55]


array([[[0.8352941 , 0.8352941 , 0.8352941 ],
        [0.89411765, 0.89411765, 0.89411765],
        [0.85882354, 0.85882354, 0.85882354],
        ...,
        [0.8901961 , 0.8901961 , 0.8901961 ],
        [0.8901961 , 0.8901961 , 0.8901961 ],
        [0.8901961 , 0.8901961 , 0.8901961 ]],

       [[0.84313726, 0.84313726, 0.84313726],
        [0.8784314 , 0.8784314 , 0.8784314 ],
        [0.827451  , 0.827451  , 0.827451  ],
        ...,
        [0.9098039 , 0.9098039 , 0.9098039 ],
        [0.90588236, 0.90588236, 0.90588236],
        [0.90588236, 0.90588236, 0.90588236]],

       [[0.85490197, 0.85490197, 0.85490197],
        [0.8666667 , 0.8666667 , 0.8666667 ],
        [0.8156863 , 0.8156863 , 0.8156863 ],
        ...,
        [0.8980392 , 0.8980392 , 0.8980392 ],
        [0.8980392 , 0.8980392 , 0.8980392 ],
        [0.89411765, 0.89411765, 0.89411765]],

       ...,

       [[0.7921569 , 0.7921569 , 0.7921569 ],
        [0.81960785, 0.81960785, 0.81960785],
        [0.8       , 0

In [ ]:
ai_labels = np.zeros(len(ai_images))
real_labels = np.ones(len(real_images))
ai_images.shape


(655, 224, 224, 3)

In [ ]:
real_images.shape

(600, 224, 224, 3)

In [ ]:
X_train = np.concatenate((ai_images, real_images), axis=0)
y_train = np.concatenate((ai_labels, real_labels), axis=0)


In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (1255, 224, 224, 3)
Shape of y_train: (1255,)


In [ ]:
y_train[:2]


array([0., 0.])

In [ ]:
y_train=y_train.reshape(-1,)

In [ ]:
y_train[:2 ]

array([0., 0.])

In [ ]:
X_train[:1]

In [ ]:
import keras
from keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(224,
                                                              224,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
num_classes = 2

model = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(2, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


40/40 [==============================] - 4s 45ms/step - loss: 0.9291 - accuracy: 0.5745
Epoch 2/30
40/40 [==============================] - 2s 43ms/step - loss: 0.6674 - accuracy: 0.5801
Epoch 3/30
40/40 [==============================] - 2s 43ms/step - loss: 0.6395 - accuracy: 0.6104
Epoch 4/30
40/40 [==============================] - 2s 47ms/step - loss: 0.6422 - accuracy: 0.6414
Epoch 5/30
40/40 [==============================] - 2s 45ms/step - loss: 0.6309 - accuracy: 0.6303
Epoch 6/30
40/40 [==============================] - 2s 42ms/step - loss: 0.6107 - accuracy: 0.6598
Epoch 7/30
40/40 [==============================] - 2s 43ms/step - loss: 0.6096 - accuracy: 0.6614
Epoch 8/30
40/40 [==============================] - 2s 42ms/step - loss: 0.5973 - accuracy: 0.6829
Epoch 9/30
40/40 [==============================] - 2s 42ms/step - loss: 0.5914 - accuracy: 0.6813
Epoch 10/30
40/40 [==============================] - 2s 42ms/step - loss: 0.5858 - accuracy: 0.6845
Epoch 11/30
40/40 [=

In [ ]:
'''cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32,32, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])'''

In [ ]:
'''cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])'''

In [ ]:
'''cnn.fit(X_train, y_train, epochs=15)'''

In [ ]:
test_images = load_and_preprocess_images_from_folder('/content/Data/Test_Images')

<ipython-input-5-aa644e78db35>:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img.thumbnail(target_size, Image.ANTIALIAS)


In [ ]:
X_test=test_images
X_test.shape

(500, 224, 224, 3)

In [ ]:
y_pred = model.predict(X_test)
y_pred[:]

16/16 [==============================] - 1s 34ms/step


array([[0.5108174 , 0.39937377],
       [0.5400308 , 0.27735117],
       [0.6939379 , 0.28918165],
       [0.7908977 , 0.19137125],
       [0.41693977, 0.5045125 ],
       [0.34154072, 0.72604996],
       [0.3441672 , 0.56564   ],
       [0.20627572, 0.6292523 ],
       [0.63398856, 0.2458406 ],
       [0.22572026, 0.62520707],
       [0.3712253 , 0.46975422],
       [0.5379785 , 0.3637912 ],
       [0.3044286 , 0.53125125],
       [0.56034654, 0.3183785 ],
       [0.4943736 , 0.6033996 ],
       [0.03406928, 0.8350109 ],
       [0.43031865, 0.39632553],
       [0.32297787, 0.44988734],
       [0.15038502, 0.77025664],
       [0.2819838 , 0.58941203],
       [0.6608932 , 0.3327361 ],
       [0.4053105 , 0.44163176],
       [0.13790864, 0.7530801 ],
       [0.44059074, 0.34526068],
       [0.58902776, 0.3771272 ],
       [0.6363519 , 0.2936355 ],
       [0.43543878, 0.35338   ],
       [0.42985868, 0.44530824],
       [0.2383144 , 0.6145223 ],
       [0.12167673, 0.6979606 ],
       [0.

In [ ]:
y_classes = [np.argmax(i) for i in y_pred]
y_classes[:]

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,


In [ ]:
labels = ['Real' if label == 1 else 'AI' for label in y_classes]
ids = list(y_classes)
print("ids:", ids[:5])
print("labels:", labels[:5])


ids: [0, 0, 0, 0, 1]
labels: ['AI', 'AI', 'AI', 'AI', 'Real']


In [ ]:
print(ids[:])
print(labels[:])

[0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 

In [ ]:
df = pd.DataFrame({'id': ids, 'label': labels})

In [ ]:
 #Save this DataFrame to CSV file
df.to_csv('output3.csv', index=False)
print("DataFrame saved to 'output.csv'")

DataFrame saved to 'output.csv'
